# CHEM 584 Machine Learning Project
## Marcus Sak

### GPT-2 Text Generation

This notebook takes in a single-column `.csv` file containing the body text of scientific articles, trains a GPT-2 model, and outputs synthetic text into a `.txt` file.

### Imports

We use [gpt-2-simple](https://github.com/minimaxir/gpt-2-simple), a Python wrapper for [OpenAI](https://openai.com/)'s [GPT-2 text generation model](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf). This requires tensorflow-gpu version 1 to be installed.

In [1]:
import os
import requests
import tensorflow as tf  # must be tensorflow 1.x
import gpt_2_simple as gpt2
from datetime import datetime

Four versions of GPT-2 have been released, with 124M, 355M, 774M, and 1558M (full-sized) hyperparameters respectively. The upper limit for one commercially available GPU is the 774M model. This notebook was run on a Grace node with a single NVIDIA Tesla V100 GPU with 16GB of memory. 

In [8]:
model_name = "774M"
if not os.path.isdir(os.path.join("models", model_name)):
    print(f"Downloading {model_name} model...")
    gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/774M/

### Final data processing 

The final part of the data processing is handled in this notebook. In particular, we remove double quotes, empty lines, and break up the text into 1024-token chunks.

In [3]:
file_name = "./preproc/text_proc.csv"

# preproc leaves double quotes, remove here
if not os.path.isfile(file_name):
    print("The file does not exist")
else:
    with open(file_name, 'r') as file:
        data = file.read().replace('\"', '').replace('\n\n', '\n')

    with open(file_name, 'w') as file:
        file.write(data)

In [7]:
all_words = []
for line in open(file_name):
    row = line.split(' ')
    all_words += list(row)

line_breaker = 1023

# generate actual input file
gpt_input = './text_proc_2.txt'
with open(gpt_input, 'w') as file:
    for index, word in enumerate(all_words):
        if not (index % line_breaker) and index:
            # every 1023 words, make a newline
            file.write(word.strip('\n')+"\n")
        else:
            file.write(word.strip('\n')+" ")

### Training GPT-2

The gpt-2-simple wrapper greatly simplifies the GPT-2 interface. Training involves specifying the input file and some keyword arguments that are relatively self-explanatory. Model training was stopped at around 12000 steps after observing that the loss had plateaued and the sample output was starting to reproduce phrases in the source text.

In [9]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=gpt_input,
              model_name='774M',
              steps=20000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=1000
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/774M/model.ckpt
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:03<00:00,  3.52s/it]


dataset has 762348 tokens
Training...
[10 | 24.11] loss=3.58 avg=3.58
[20 | 34.16] loss=3.01 avg=3.29
[30 | 44.22] loss=3.59 avg=3.39
[40 | 54.30] loss=2.75 avg=3.23
[50 | 64.39] loss=3.02 avg=3.19
[60 | 74.47] loss=2.50 avg=3.07
[70 | 84.57] loss=2.82 avg=3.03
[80 | 94.66] loss=2.76 avg=3.00
[90 | 104.75] loss=2.82 avg=2.98
[100 | 114.85] loss=2.94 avg=2.97
[110 | 124.96] loss=2.74 avg=2.95
[120 | 135.05] loss=2.86 avg=2.94
[130 | 145.14] loss=2.88 avg=2.94
[140 | 155.24] loss=2.05 avg=2.87
[150 | 165.33] loss=2.81 avg=2.87
[160 | 175.41] loss=2.31 avg=2.83
[170 | 185.51] loss=2.29 avg=2.79
[180 | 195.60] loss=2.90 avg=2.80
[190 | 205.68] loss=2.29 avg=2.77
[200 | 215.77] loss=2.95 avg=2.78
======== SAMPLE 1 ========
 stoivity, the incorporation or addition of any metal ions to the arylation products will likely result in enhanced rate and enantioselectivity. To achieve higher enantiocontrol, we explored the capacity of metal ions to influence the rates of aldination in addition to re

...

[10410 | 11583.60] loss=0.07 avg=0.13
[10420 | 11593.65] loss=0.05 avg=0.13
[10430 | 11603.72] loss=0.11 avg=0.13
[10440 | 11613.78] loss=0.13 avg=0.13
[10450 | 11623.84] loss=0.13 avg=0.13
[10460 | 11633.91] loss=0.11 avg=0.13
[10470 | 11643.98] loss=0.05 avg=0.12
[10480 | 11654.04] loss=0.07 avg=0.12
[10490 | 11664.11] loss=0.04 avg=0.12
[10500 | 11674.18] loss=0.09 avg=0.12
[10510 | 11684.25] loss=0.09 avg=0.12
[10520 | 11694.32] loss=0.14 avg=0.12
[10530 | 11704.39] loss=0.06 avg=0.12
[10540 | 11714.46] loss=0.07 avg=0.12
[10550 | 11724.53] loss=0.14 avg=0.12
[10560 | 11734.61] loss=0.07 avg=0.12
[10570 | 11744.69] loss=0.12 avg=0.12
[10580 | 11754.76] loss=0.29 avg=0.12
[10590 | 11764.84] loss=0.14 avg=0.12
[10600 | 11774.92] loss=0.18 avg=0.12
======== SAMPLE 1 ========
 frame-mediated reaction of aldehydes at the 2-position of acylimines is a much more challenging objective, and also requires a much longer reaction time. In fact, examinations of the peptide-based catalysts for t

[10810 | 12025.42] loss=0.09 avg=0.12
[10820 | 12035.47] loss=0.10 avg=0.12
[10830 | 12045.54] loss=0.17 avg=0.12
[10840 | 12055.61] loss=0.26 avg=0.12
[10850 | 12065.68] loss=0.07 avg=0.12
[10860 | 12075.75] loss=0.06 avg=0.12
[10870 | 12085.83] loss=0.16 avg=0.12
[10880 | 12095.91] loss=0.16 avg=0.12
[10890 | 12105.98] loss=0.15 avg=0.12
[10900 | 12116.07] loss=0.10 avg=0.12
[10910 | 12126.15] loss=0.09 avg=0.12
[10920 | 12136.22] loss=0.12 avg=0.12
[10930 | 12146.29] loss=0.13 avg=0.12
[10940 | 12156.37] loss=0.09 avg=0.12
[10950 | 12166.45] loss=0.17 avg=0.12
[10960 | 12176.53] loss=0.10 avg=0.12
[10970 | 12186.60] loss=0.15 avg=0.12
[10980 | 12196.68] loss=0.09 avg=0.12
[10990 | 12206.76] loss=0.08 avg=0.12
[11000 | 12216.84] loss=0.06 avg=0.12
Saving checkpoint/run1/model-11000
======== SAMPLE 1 ========
 blocks, both of which contain a dialkylamide motif, and a 2, 3-dialkylamide scaffold. We then designed structural panels around these structural motifs using DFT calculations. W

[11210 | 12471.24] loss=0.10 avg=0.12
[11220 | 12481.31] loss=0.15 avg=0.12
[11230 | 12491.38] loss=0.08 avg=0.12
[11240 | 12501.44] loss=0.04 avg=0.12
[11250 | 12511.52] loss=0.13 avg=0.12
[11260 | 12521.60] loss=0.09 avg=0.12
[11270 | 12531.67] loss=0.08 avg=0.12
[11280 | 12541.75] loss=0.07 avg=0.11
[11290 | 12551.83] loss=0.12 avg=0.11
[11300 | 12561.91] loss=0.24 avg=0.12
[11310 | 12571.99] loss=0.19 avg=0.12
[11320 | 12582.07] loss=0.09 avg=0.12
[11330 | 12592.14] loss=0.15 avg=0.12
[11340 | 12602.22] loss=0.27 avg=0.12
[11350 | 12612.29] loss=0.10 avg=0.12
[11360 | 12622.36] loss=0.09 avg=0.12
[11370 | 12632.44] loss=0.11 avg=0.12
[11380 | 12642.52] loss=0.10 avg=0.12
[11390 | 12652.59] loss=0.14 avg=0.12
[11400 | 12662.67] loss=0.10 avg=0.12
======== SAMPLE 1 ========
ines
Dalcations. Following reaction of 25 for 10 min, HPLC and LCMS analysis revealed 86 remaining starting material along with minor mono-glucosyl product peaks, further supporting Tyr as the site of glycosylatio

[11610 | 12913.32] loss=0.09 avg=0.12
[11620 | 12923.38] loss=0.14 avg=0.12
[11630 | 12933.44] loss=0.10 avg=0.12
[11640 | 12943.50] loss=0.06 avg=0.12
[11650 | 12953.58] loss=0.10 avg=0.11
[11660 | 12963.67] loss=0.09 avg=0.11
[11670 | 12973.74] loss=0.09 avg=0.11
[11680 | 12983.82] loss=0.12 avg=0.11
[11690 | 12993.90] loss=0.10 avg=0.11
[11700 | 13003.98] loss=0.10 avg=0.11
[11710 | 13014.06] loss=0.08 avg=0.11
[11720 | 13024.14] loss=0.09 avg=0.11
[11730 | 13034.21] loss=0.07 avg=0.11
[11740 | 13044.29] loss=0.12 avg=0.11
[11750 | 13054.37] loss=0.17 avg=0.11
[11760 | 13064.45] loss=0.08 avg=0.11
[11770 | 13074.52] loss=0.13 avg=0.11
[11780 | 13084.60] loss=0.11 avg=0.11
[11790 | 13094.67] loss=0.07 avg=0.11
[11800 | 13104.75] loss=0.15 avg=0.11
======== SAMPLE 1 ========
 technological 1g and P14. Given the magnitude of the effect, P19 was found to be the more selective catalyst, providing 5:1d in 48 h at 25C. Higher rates of racemization under more mild conditions were obtained u

[12010 | 13359.07] loss=0.11 avg=0.11
[12020 | 13369.12] loss=0.16 avg=0.11
[12030 | 13379.18] loss=0.11 avg=0.11
[12040 | 13389.25] loss=0.08 avg=0.11
interrupted
Saving checkpoint/run1/model-12046


### Text generation

We passed in a `prefix` to the generate function to force the text to start with a given character sequence. We chose prefixes that might prompt the model to generate new ideas, such as "Photoredox catalysis can be applied to...". 

The `batch_size` option allows the generation of multiple samples in parallel, affording a speedup. The `top_p` option limits the generated guesses to a cumulative probability.

In [20]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}_0.85.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=250,
                          prefix="Peptide catalysis can be applied to",
                      temperature=0.85,
                      nsamples=200,
                      batch_size=10
                      )